# I. Import libraries

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

# II. Connect to the data

In [3]:
spark = SparkSession \
.builder \
.appName("Wrangling covid19 data") \
.getOrCreate()

In [4]:
path = 'data/jantojun2020.csv'
data = spark.read.csv(path, header=True)

# III. Assess the data

In [5]:
data.count()

2745847

In [6]:
data.columns

['YEAR',
 'QUARTER',
 'MONTH',
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
 'FL_DATE',
 'MKT_UNIQUE_CARRIER',
 'MKT_CARRIER_FL_NUM',
 'TAIL_NUM',
 'ORIGIN',
 'ORIGIN_CITY_NAME',
 'ORIGIN_STATE_ABR',
 'ORIGIN_STATE_NM',
 'DEST',
 'DEST_CITY_NAME',
 'DEST_STATE_ABR',
 'DEST_STATE_NM',
 'CRS_DEP_TIME',
 'DEP_TIME',
 'DEP_DELAY',
 'DEP_DELAY_NEW',
 'DEP_DEL15',
 'DEP_DELAY_GROUP',
 'DEP_TIME_BLK',
 'TAXI_OUT',
 'WHEELS_OFF',
 'WHEELS_ON',
 'TAXI_IN',
 'CRS_ARR_TIME',
 'ARR_TIME',
 'ARR_DELAY',
 'ARR_DELAY_NEW',
 'ARR_DEL15',
 'ARR_DELAY_GROUP',
 'ARR_TIME_BLK',
 'CANCELLED',
 'CANCELLATION_CODE',
 'CRS_ELAPSED_TIME',
 'ACTUAL_ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'DISTANCE_GROUP',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DELAY',
 'SECURITY_DELAY',
 'LATE_AIRCRAFT_DELAY']

# IV. Wrangling data
## 1. Split data to dim tables

### 1.1 Create port_loc dataframe

In [7]:
def create_port_loc_df(path):
    df = spark.read.csv(path, header=True)
    for column in df.columns:
        df = df.withColumnRenamed(column, column.lower())
    port_loc_df = df.select('origin', 'origin_city_name', 'origin_state_abr').dropDuplicates()
    port_loc_df = port_loc_df.withColumn('origin_city_name', split(port_loc_df['origin_city_name'], ',').getItem(0))
    port_loc_df.toPandas().to_csv('data/port_loc.csv', index=False)

In [8]:
create_port_loc_df(path)

In [9]:
# test
port_loc_df = spark.read.csv('data/port_loc.csv', header=True)
port_loc_df.show(10)

+------+----------------+----------------+
|origin|origin_city_name|origin_state_abr|
+------+----------------+----------------+
|   SAF|        Santa Fe|              NM|
|   MSP|     Minneapolis|              MN|
|   TUL|           Tulsa|              OK|
|   DBQ|         Dubuque|              IA|
|   LFT|       Lafayette|              LA|
|   ROW|         Roswell|              NM|
|   PIT|      Pittsburgh|              PA|
|   SLN|          Salina|              KS|
|   EAU|      Eau Claire|              WI|
|   DCA|      Washington|              VA|
+------+----------------+----------------+
only showing top 10 rows



### 1.2 Create states dataframe

In [10]:
def create_states_df(path):
    df = spark.read.csv(path, header=True)
    for column in df.columns:
        df = df.withColumnRenamed(column, column.lower())
    state_df = df.select('origin_state_abr', 'origin_state_nm').dropDuplicates()
    state_df.toPandas().to_csv('data/states.csv', index=False)

In [11]:
create_states_df(path)

In [12]:
# test
states_df = spark.read.csv('data/states.csv', header=True)
states_df.show(10)

+----------------+-------------------+
|origin_state_abr|    origin_state_nm|
+----------------+-------------------+
|              VI|U.S. Virgin Islands|
|              MT|            Montana|
|              NC|     North Carolina|
|              MD|           Maryland|
|              CO|           Colorado|
|              CT|        Connecticut|
|              IL|           Illinois|
|              WY|            Wyoming|
|              NJ|         New Jersey|
|              LA|          Louisiana|
+----------------+-------------------+
only showing top 10 rows



In [13]:
states_df.count()

52

### 1.3 Create airline dataframe

In [32]:
def create_airline_df(path1,path2):
        """
        Function: Generate and create airlines table code
        param:
            - path1: txt file
            - path2: dataset file
        output: airline.csv file stored in data folder
        """
        df = spark.read.csv(path2, header=True)
        df2 = df.select("MKT_UNIQUE_CARRIER","MKT_CARRIER_FL_NUM","ORIGIN","DEST","TAIL_NUM").dropDuplicates()
        with open(path1) as f:
            content = f.readlines()
            content = [x.strip() for x in content]
            airline = content[10:20]
            splitted_airline = [c.split(":") for c in airline]
            c_airline = [x[0].replace("'","").strip() for x in splitted_airline]
            airline_name = [x[1].replace("'","").strip() for x in splitted_airline]
            airline_df = spark.createDataFrame(zip(c_airline, airline_name), schema=['c_airline', 'airline_name'])
            airline_df = airline_df.join(df2,airline_df.c_airline == df2.MKT_UNIQUE_CARRIER,"inner")\
                        .drop("MKT_UNIQUE_CARRIER","ORIGIN","DEST")
            return airline_df.toPandas().to_csv("data/airline.csv", index=False)

In [37]:
path_txt = 'data/ColumnDescriptions.txt'
path_csv = 'data/jantojun2020.csv'
create_airline_df(path1, path2)

In [34]:
# test
airline_df = spark.read.csv('data/airline.csv', header=True)
airline_df.show(10)

+---------+---------------+------------------+--------+
|c_airline|   airline_name|MKT_CARRIER_FL_NUM|TAIL_NUM|
+---------+---------------+------------------+--------+
|       NK|Spirit Airlines|              1009|  N624NK|
|       NK|Spirit Airlines|               103|  N654NK|
|       NK|Spirit Airlines|              1069|  N913NK|
|       NK|Spirit Airlines|               109|  N507NK|
|       NK|Spirit Airlines|              1247|  N648NK|
|       NK|Spirit Airlines|              1360|  N629NK|
|       NK|Spirit Airlines|              1400|  N602NK|
|       NK|Spirit Airlines|              1440|  N915NK|
|       NK|Spirit Airlines|              1520|  N672NK|
|       NK|Spirit Airlines|              1782|  N627NK|
+---------+---------------+------------------+--------+
only showing top 10 rows



### 1.4 Create distance_group dataframe

In [17]:
data.select('distance', 'distance_group').show(10)

+--------+--------------+
|distance|distance_group|
+--------+--------------+
|     363|             2|
|     363|             2|
|     333|             2|
|     333|             2|
|     333|             2|
|     333|             2|
|     333|             2|
|     390|             2|
|     390|             2|
|     390|             2|
+--------+--------------+
only showing top 10 rows



By observing the table above, we can clearly figure out there are some wrong values in this dataframe: it should be 1 for those distances (Based on the explanation in ColumnDescriptions.txt). Therefore, we will fix it first.
##### Fix the wrong distance_group values

In [18]:
# use floor() to get the integer division
data = data.withColumn('distance_group', floor(data['distance'].cast('int')/250))
# test
data.select('distance', 'distance_group').show(10)

+--------+--------------+
|distance|distance_group|
+--------+--------------+
|     363|             1|
|     363|             1|
|     333|             1|
|     333|             1|
|     333|             1|
|     333|             1|
|     333|             1|
|     390|             1|
|     390|             1|
|     390|             1|
+--------+--------------+
only showing top 10 rows



##### Create the wanted dataframe

In [19]:
data.select('distance_group').describe().show()

+-------+------------------+
|summary|    distance_group|
+-------+------------------+
|  count|           2745847|
|   mean|  2.50195659117205|
| stddev|2.2411625220911175|
|    min|                 0|
|    max|                20|
+-------+------------------+



Because the min and the max values of the distance_group are 0 and 20, respectively,  we choose the range from 0 to 22 for this distance_group data frame.

In [20]:
def create_distance_group():
    data = []
    for i in range(23):
        data.append([i, "{} <= distance < {}".format(i * 250, (i + 1) * 250)])
        
    df = pd.DataFrame(data=data, columns=['distance_group', 'distance_range(miles)'])
    df.to_csv('data/distance_group.csv', index=False)    

In [21]:
create_distance_group()

In [22]:
# test
distance_group_df = spark.read.csv('data/distance_group.csv', header=True)
distance_group_df.show(10, truncate=False)

+--------------+-----------------------+
|distance_group|distance_range(miles)  |
+--------------+-----------------------+
|0             |0 <= distance < 250    |
|1             |250 <= distance < 500  |
|2             |500 <= distance < 750  |
|3             |750 <= distance < 1000 |
|4             |1000 <= distance < 1250|
|5             |1250 <= distance < 1500|
|6             |1500 <= distance < 1750|
|7             |1750 <= distance < 2000|
|8             |2000 <= distance < 2250|
|9             |2250 <= distance < 2500|
+--------------+-----------------------+
only showing top 10 rows



### 1.5 Create cancelation dataframe

In [42]:
def create_cancelation_df(path):
        """
        Function: Generate and create Cancelation_code table:
        param: Path of datafile
        input: .txt file
        output: cancel.csv file stored in data folder
        """
        import re
        with open(path) as f:
            content = f.readlines()
            content = [x.strip() for x in content]
            cancel = [re.search('\(([^)]+)', content[49]).group(1)][0].split(",")
            splitted_cancel = [c.split(":") for c in cancel]
            c_cancel = [x[0].replace("'","").strip() for x in splitted_cancel]
            cancel_des= [x[1].replace("'","").strip() for x in splitted_cancel]
            cancel_df = pd.DataFrame({"c_cancel" : c_cancel, "cancel_des": cancel_des})
            return cancel_df.to_csv("data/airline.csv", index=False)

In [43]:
create_cancelation_df(path_txt)

In [48]:
# test
cancel_df = spark.read.csv('data/airline.csv', header=True)
cancel_df.show(truncate=False)

+--------+------------------------+
|c_cancel|cancel_des              |
+--------+------------------------+
|A       |Carrier                 |
|B       |Weather                 |
|C       |National Aviation System|
|D       |Security                |
+--------+------------------------+



### 1.6 Create delay_group dataframe

In [45]:
def create_delay_group():
        """
        function
        """
        data = []
        for i in range(-1,188):
            if i == -1:
                data.append([-1,"Early"])
            elif i == 0:
                data.append([0,"On Time"])
            else:
                data.append([i, "{} <= delay time < {}".format(i * 15, (i + 1) * 15)])

        df = pd.DataFrame(data=data, columns=['delay_group', 'delay_time_range(minutes)'])
        df.to_csv('data/delay_group.csv', index=False)

In [46]:
create_delay_group()

In [47]:
# test
delay_group_df = spark.read.csv('data/delay_group.csv', header=True)
delay_group_df.show(10, truncate=False)

+-----------+-------------------------+
|delay_group|delay_time_range(minutes)|
+-----------+-------------------------+
|-1         |Early                    |
|0          |On Time                  |
|1          |15 <= delay time < 30    |
|2          |30 <= delay time < 45    |
|3          |45 <= delay time < 60    |
|4          |60 <= delay time < 75    |
|5          |75 <= delay time < 90    |
|6          |90 <= delay time < 105   |
|7          |105 <= delay time < 120  |
|8          |120 <= delay time < 135  |
+-----------+-------------------------+
only showing top 10 rows



## 2. Check null values
### 2.1 airline dataframe

In [22]:
airline_df.show(10)

+---------+------------------+
|c_airline|      airline_name|
+---------+------------------+
|       AA| American Airlines|
|       AS|   Alaska Airlines|
|       B6|           JetBlue|
|       DL|   Delta Air Lines|
|       F9| Frontier Airlines|
|       G4|     Allegiant Air|
|       HA| Hawaiian Airlines|
|       NK|   Spirit Airlines|
|       UA|   United Airlines|
|       WN|Southwest Airlines|
+---------+------------------+



In [23]:
airline_df.filter(col('c_airline').isNull()).show()

+---------+------------+
|c_airline|airline_name|
+---------+------------+
+---------+------------+



There is no null value for column c_airline in airline dataframe
### 2.2 distance_group dataframe

In [24]:
distance_group_df.show(10, truncate=False)

+--------------+-----------------------+
|distance_group|distance_range(miles)  |
+--------------+-----------------------+
|0             |0 <= distance < 250    |
|1             |250 <= distance < 500  |
|2             |500 <= distance < 750  |
|3             |750 <= distance < 1000 |
|4             |1000 <= distance < 1250|
|5             |1250 <= distance < 1500|
|6             |1500 <= distance < 1750|
|7             |1750 <= distance < 2000|
|8             |2000 <= distance < 2250|
|9             |2250 <= distance < 2500|
+--------------+-----------------------+
only showing top 10 rows



In [25]:
distance_group_df.filter(col('distance_group').isNull()).show()

+--------------+---------------------+
|distance_group|distance_range(miles)|
+--------------+---------------------+
+--------------+---------------------+



There is no null value for column distance_group in distance_group dataframe
### 2.3 States dataframe

In [26]:
states_df.show(10)

+----------------+-------------------+
|origin_state_abr|    origin_state_nm|
+----------------+-------------------+
|              VI|U.S. Virgin Islands|
|              MT|            Montana|
|              NC|     North Carolina|
|              MD|           Maryland|
|              CO|           Colorado|
|              CT|        Connecticut|
|              IL|           Illinois|
|              WY|            Wyoming|
|              NJ|         New Jersey|
|              LA|          Louisiana|
+----------------+-------------------+
only showing top 10 rows



In [27]:
states_df.filter(col('origin_state_abr').isNull()).show()

+----------------+---------------+
|origin_state_abr|origin_state_nm|
+----------------+---------------+
+----------------+---------------+



There is no null value for column origin_state_abr in states dataframe
### 2.4 port_loc dataframe

In [28]:
port_loc_df.show(10)

+------+----------------+----------------+
|origin|origin_city_name|origin_state_abr|
+------+----------------+----------------+
|   SAF|        Santa Fe|              NM|
|   MSP|     Minneapolis|              MN|
|   TUL|           Tulsa|              OK|
|   DBQ|         Dubuque|              IA|
|   LFT|       Lafayette|              LA|
|   ROW|         Roswell|              NM|
|   PIT|      Pittsburgh|              PA|
|   SLN|          Salina|              KS|
|   EAU|      Eau Claire|              WI|
|   DCA|      Washington|              VA|
+------+----------------+----------------+
only showing top 10 rows



In [29]:
port_loc_df.filter(col('origin').isNull()).show()

+------+----------------+----------------+
|origin|origin_city_name|origin_state_abr|
+------+----------------+----------------+
+------+----------------+----------------+



There is no null value for column origin in port_loc dataframe

## 3. Create fact table

In [30]:
data.select('year', 'quarter', 'month', 'day_of_month','day_of_week', 'fl_date').show(10)

+----+-------+-----+------------+-----------+--------+
|year|quarter|month|day_of_month|day_of_week| fl_date|
+----+-------+-----+------------+-----------+--------+
|2020|      1|    1|           1|          3|1/1/2020|
|2020|      1|    1|           1|          3|1/1/2020|
|2020|      1|    1|           1|          3|1/1/2020|
|2020|      1|    1|           1|          3|1/1/2020|
|2020|      1|    1|           1|          3|1/1/2020|
|2020|      1|    1|           1|          3|1/1/2020|
|2020|      1|    1|           1|          3|1/1/2020|
|2020|      1|    1|           1|          3|1/1/2020|
|2020|      1|    1|           1|          3|1/1/2020|
|2020|      1|    1|           1|          3|1/1/2020|
+----+-------+-----+------------+-----------+--------+
only showing top 10 rows



In [31]:
fact_df = data.withColumn("flight_date",concat_ws("-",col("year"),col("month"),col("day_of_month")).cast("date"))
# test
fact_df.select('fl_date', 'flight_date').show(10)

+--------+-----------+
| fl_date|flight_date|
+--------+-----------+
|1/1/2020| 2020-01-01|
|1/1/2020| 2020-01-01|
|1/1/2020| 2020-01-01|
|1/1/2020| 2020-01-01|
|1/1/2020| 2020-01-01|
|1/1/2020| 2020-01-01|
|1/1/2020| 2020-01-01|
|1/1/2020| 2020-01-01|
|1/1/2020| 2020-01-01|
|1/1/2020| 2020-01-01|
+--------+-----------+
only showing top 10 rows



##### Drop column year, quarter, month, day_of_month, day_of_week, fl_date

In [32]:
fact_df = fact_df.drop('year', 'quarter', 'month', 'day_of_month', 'day_of_week', 'fl_date')
fact_df.printSchema()

root
 |-- MKT_UNIQUE_CARRIER: string (nullable = true)
 |-- MKT_CARRIER_FL_NUM: string (nullable = true)
 |-- TAIL_NUM: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY_NAME: string (nullable = true)
 |-- ORIGIN_STATE_ABR: string (nullable = true)
 |-- ORIGIN_STATE_NM: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY_NAME: string (nullable = true)
 |-- DEST_STATE_ABR: string (nullable = true)
 |-- DEST_STATE_NM: string (nullable = true)
 |-- CRS_DEP_TIME: string (nullable = true)
 |-- DEP_TIME: string (nullable = true)
 |-- DEP_DELAY: string (nullable = true)
 |-- DEP_DELAY_NEW: string (nullable = true)
 |-- DEP_DEL15: string (nullable = true)
 |-- DEP_DELAY_GROUP: string (nullable = true)
 |-- DEP_TIME_BLK: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- CRS_ARR_TIME: string (nullable =

##### Drop columns related to origin and destination

In [33]:
fact_df = fact_df.drop('origin_city_name', 'origin_state_abr', 'origin_state_nm',\
                       'dest_city_name', 'dest_state_abr', 'dest_state_nm')
fact_df.printSchema()

root
 |-- MKT_UNIQUE_CARRIER: string (nullable = true)
 |-- MKT_CARRIER_FL_NUM: string (nullable = true)
 |-- TAIL_NUM: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: string (nullable = true)
 |-- DEP_TIME: string (nullable = true)
 |-- DEP_DELAY: string (nullable = true)
 |-- DEP_DELAY_NEW: string (nullable = true)
 |-- DEP_DEL15: string (nullable = true)
 |-- DEP_DELAY_GROUP: string (nullable = true)
 |-- DEP_TIME_BLK: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- ARR_TIME: string (nullable = true)
 |-- ARR_DELAY: string (nullable = true)
 |-- ARR_DELAY_NEW: string (nullable = true)
 |-- ARR_DEL15: string (nullable = true)
 |-- ARR_DELAY_GROUP: string (nullable = true)
 |-- ARR_TIME_BLK: string (nullable = true)
 |-- CANCELLED:

In [46]:
check_df = fact_df.select('MKT_UNIQUE_CARRIER','MKT_CARRIER_FL_NUM','tail_num').dropDuplicates()
check_df.show()

+------------------+------------------+--------+
|MKT_UNIQUE_CARRIER|MKT_CARRIER_FL_NUM|tail_num|
+------------------+------------------+--------+
|                WN|              6260|  N787SA|
|                WN|              4789|  N7717D|
|                WN|               665|  N969WN|
|                WN|              1310|  N492WN|
|                WN|              2553|  N924WN|
|                WN|              2585|  N966WN|
|                WN|               701|  N7828A|
|                WN|               971|  N7710A|
|                WN|               360|  N297WN|
|                WN|               972|  N775SW|
|                WN|               878|  N7853B|
|                WN|              1786|  N933WN|
|                WN|              4050|  N209WN|
|                WN|              1266|  N476WN|
|                WN|               450|  N8578Q|
|                WN|               245|  N269WN|
|                WN|              1166|  N8523W|
|                WN|

In [47]:
check_df.filter(col('tail_num')=='N787SA').filter(col('MKT_CARRIER_FL_NUM')=='6260').dropDuplicates().show()

+------------------+------------------+--------+
|MKT_UNIQUE_CARRIER|MKT_CARRIER_FL_NUM|tail_num|
+------------------+------------------+--------+
|                WN|              6260|  N787SA|
+------------------+------------------+--------+



In [48]:
data.select('origin', 'dest').filter(col('tail_num')=='N787SA').filter(col('MKT_CARRIER_FL_NUM')=='6260').dropDuplicates().show()

+------+----+
|origin|dest|
+------+----+
|   RNO| DEN|
|   MCI| MKE|
|   LAS| RNO|
|   DEN| MCI|
+------+----+



In [ ]:
data.select('fl_date', 'CRS_DEP_TIMe'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     